In [1]:
# Install required packages
# mamba install -c conda-forge langchain langchain-text-splitters langchain-openai langchain-community wikipedia pydantic nbformat nbconvert
%pip install -q langchain langchain-text-splitters langchain-openai langchain-community wikipedia pydantic nbformat

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import json
import hashlib
import time
import sys
from datetime import datetime
from dataclasses import dataclass, field, asdict
from typing import Optional
from urllib.parse import quote

import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell, new_raw_cell
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# === Content ID (CID) Functions ===

def compute_cid(content: str) -> str:
    """Compute SHA256 content ID for a string."""
    return hashlib.sha256(content.encode('utf-8')).hexdigest()

def make_signature(cell_num: int, cell_type: str, cid: str, from_cid: str) -> dict:
    """Create a signature dict for a generated cell."""
    return {
        "cell": cell_num,
        "type": cell_type,
        "cid": cid,
        "from_cid": from_cid,
    }

def parse_signature(raw_content: str) -> Optional[dict]:
    """Parse a signature from raw cell content. Returns None if not a valid signature."""
    try:
        data = json.loads(raw_content.strip())
        if all(k in data for k in ("cell", "type", "cid", "from_cid")):
            return data
    except (json.JSONDecodeError, TypeError):
        pass
    return None

def extract_signatures(notebook) -> dict:
    """Extract all signatures from a notebook, keyed by cell number."""
    signatures = {}
    for cell in notebook.cells:
        if cell.cell_type == 'raw':
            sig = parse_signature(cell.source)
            if sig:
                signatures[sig["cell"]] = sig
    return signatures

# === Timeout Configuration ===
# Note: SIGALRM doesn't work for blocking I/O (HTTP requests)
# Instead, we use the timeout parameter on the ChatOpenAI client

CELL_TIMEOUT_SECONDS = 60  # 1 minute per cell max
MAX_ITERATIONS = 150     # Max tools calls per cell processing

def log_progress(msg: str, end="\n"):
    """Print with immediate flush for real-time progress."""
    print(msg, end=end, flush=True)

print("Dependencies loaded (with CID support)")
print(f"Cell timeout: {CELL_TIMEOUT_SECONDS}s")

Dependencies loaded (with CID support)
Cell timeout: 60s


## Configuration

In [ ]:
# Pipeline configuration
ARTICLE_TITLE = "Albert Einstein"
OUTPUT_DIR = "data"
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 128

# Continue from a previous run (set to None for fresh run)
# Example: "data/albert_einstein/20241218_143022"
CONTINUE_FROM_RUN = "data/albert_einstein/20251218_202954"

# LLM configuration (shared across stages)
LLM_CONFIG = {
    "provider": "lm_studio",  # or "openai"
    "model": "qwen/qwen3-coder-30b",
    "temperature": 1,
    "base_url": os.environ.get("LM_STUDIO_BASE_URL", "http://host.docker.internal:1234/v1"),
}

# Generate run timestamp and article-specific output directory
# Structure: data/{article_slug}/{timestamp}/
RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
article_slug = ARTICLE_TITLE.lower().replace(' ', '_')
RUN_OUTPUT_DIR = os.path.join(OUTPUT_DIR, article_slug, RUN_TIMESTAMP)

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(RUN_OUTPUT_DIR, exist_ok=True)

# Copy files from previous run if specified
if CONTINUE_FROM_RUN:
    import shutil
    if os.path.isdir(CONTINUE_FROM_RUN):
        print(f"Continuing from previous run: {CONTINUE_FROM_RUN}")
        copied_files = []
        for filename in os.listdir(CONTINUE_FROM_RUN):
            src = os.path.join(CONTINUE_FROM_RUN, filename)
            dst = os.path.join(RUN_OUTPUT_DIR, filename)
            if os.path.isfile(src):
                shutil.copy2(src, dst)
                copied_files.append(filename)
        print(f"  Copied {len(copied_files)} files: {', '.join(copied_files)}")
        print(f"  Existing cells with matching CIDs will be skipped")
    else:
        print(f"WARNING: Previous run not found: {CONTINUE_FROM_RUN}")
        print(f"  Starting fresh run instead")

print(f"Pipeline configured for: {ARTICLE_TITLE}")
print(f"Output directory: {RUN_OUTPUT_DIR}")
print(f"Run timestamp: {RUN_TIMESTAMP}")

Pipeline configured for: Albert Einstein
Output directory: data/albert_einstein/20251218_202954
Run timestamp: 20251218_202954


## Entity Registry

Manages entity identity across chunks with stable URIs derived from source URL.

In [4]:
@dataclass
class EntityRegistry:
    """Tracks entities with stable IDs derived from source URL."""
    source_url: str
    entities: dict = field(default_factory=dict)  # normalized_key -> entity
    aliases: dict = field(default_factory=dict)   # alias -> canonical_key
    
    def normalize_key(self, label: str) -> str:
        """Create consistent key from entity label."""
        return re.sub(r'[^a-z0-9]+', '_', label.lower().strip()).strip('_')
    
    def generate_uri(self, entity_type: str, label: str) -> str:
        """Generate URI based on source URL with fragment identifier."""
        key = self.normalize_key(label)
        # Use source URL as base, add fragment for entity
        return f"{self.source_url}#{entity_type.lower()}_{key}"
    
    def generate_id(self, entity_type: str, label: str) -> str:
        """Generate local ID for internal reference."""
        key = self.normalize_key(label)
        return f"{entity_type.lower()}_{key}"
    
    def register(self, label: str, entity_type: str, description: str = "",
                 aliases: list = None, source_chunk: int = None) -> str:
        """Register or update an entity, return canonical ID."""
        key = self.normalize_key(label)
        entity_id = self.generate_id(entity_type, label)
        entity_uri = self.generate_uri(entity_type, label)
        
        if key not in self.entities:
            self.entities[key] = {
                "id": entity_id,
                "uri": entity_uri,
                "label": label,
                "type": entity_type,
                "descriptions": [description] if description else [],
                "source_chunks": [source_chunk] if source_chunk is not None else [],
                "aliases": list(aliases or []),
            }
        else:
            existing = self.entities[key]
            if description and description not in existing["descriptions"]:
                existing["descriptions"].append(description)
            if source_chunk is not None and source_chunk not in existing["source_chunks"]:
                existing["source_chunks"].append(source_chunk)
            if aliases:
                existing["aliases"] = list(set(existing["aliases"]) | set(aliases))
        
        # Register aliases
        for alias in (aliases or []):
            self.aliases[self.normalize_key(alias)] = key
        
        return entity_id
    
    def lookup(self, label: str) -> Optional[dict]:
        """Find entity by label or alias."""
        key = self.normalize_key(label)
        canonical_key = self.aliases.get(key, key)
        return self.entities.get(canonical_key)
    
    def to_json(self) -> str:
        """Serialize registry to JSON."""
        return json.dumps({
            "source_url": self.source_url,
            "entities": self.entities,
            "aliases": self.aliases,
        }, indent=2)
    
    @classmethod
    def from_json(cls, json_str: str) -> 'EntityRegistry':
        """Deserialize registry from JSON."""
        data = json.loads(json_str)
        registry = cls(source_url=data["source_url"])
        registry.entities = data["entities"]
        registry.aliases = data["aliases"]
        return registry

print("EntityRegistry class defined")

EntityRegistry class defined


## Section Hierarchy Parser

Extracts Wikipedia section structure for breadcrumb context.

In [5]:
def extract_section_hierarchy(content: str) -> list[dict]:
    """Parse Wikipedia == headers == into hierarchical structure with positions."""
    header_pattern = re.compile(r'^(={2,6})\s*(.+?)\s*\1\s*$', re.MULTILINE)
    
    sections = []
    current_path = []  # Stack of (level, title)
    
    for match in header_pattern.finditer(content):
        level = len(match.group(1))  # Number of '=' signs
        title = match.group(2).strip()
        
        # Pop stack until we're at parent level
        while current_path and current_path[-1][0] >= level:
            current_path.pop()
        
        current_path.append((level, title))
        breadcrumb = " > ".join(t for _, t in current_path)
        
        sections.append({
            "level": level,
            "title": title,
            "breadcrumb": breadcrumb,
            "start_pos": match.start(),
            "end_pos": match.end(),
        })
    
    return sections


def get_section_context(position: int, sections: list[dict], article_title: str) -> dict:
    """Find the section context for a given character position."""
    active_section = {
        "title": "Introduction",
        "breadcrumb": "Introduction",
        "level": 1,
    }
    
    for section in sections:
        if section["start_pos"] <= position:
            active_section = section
        else:
            break
    
    return {
        "section_title": active_section["title"],
        "breadcrumb": f"{article_title} > {active_section['breadcrumb']}",
    }

print("Section hierarchy parser defined")

Section hierarchy parser defined


## Fetch Wikipedia Content

In [6]:
# Fetch Wikipedia article
loader = WikipediaLoader(query=ARTICLE_TITLE, load_max_docs=1, doc_content_chars_max=100000)
docs = loader.load()

if not docs:
    raise ValueError(f"Could not fetch article: {ARTICLE_TITLE}")

raw_content = docs[0].page_content
metadata = docs[0].metadata

# Construct source URL and provenance
source_url = f"https://en.wikipedia.org/wiki/{quote(ARTICLE_TITLE.replace(' ', '_'))}"

provenance = {
    "source_url": source_url,
    "article_title": ARTICLE_TITLE,
    "fetched_at": datetime.now().isoformat(),
    "content_length": len(raw_content),
    # Wikipedia license - standard for all Wikipedia content
    "license": "CC BY-SA 4.0",
    "license_url": "https://creativecommons.org/licenses/by-sa/4.0/",
    "attribution": "Wikipedia contributors",
}

print(f"Fetched: {ARTICLE_TITLE}")
print(f"Source URL: {source_url}")
print(f"Content length: {len(raw_content)} characters")
print(f"License: {provenance['license']}")

Fetched: Albert Einstein
Source URL: https://en.wikipedia.org/wiki/Albert_Einstein
Content length: 87959 characters
License: CC BY-SA 4.0


## Create Contextual Chunks

In [7]:
# Parse section hierarchy
sections = extract_section_hierarchy(raw_content)
print(f"Found {len(sections)} sections")

# Split into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)
raw_chunks = splitter.split_text(raw_content)
print(f"Split into {len(raw_chunks)} chunks")

# Add context to each chunk
@dataclass
class ContextualChunk:
    content: str
    chunk_index: int
    total_chunks: int
    breadcrumb: str
    section_title: str
    char_start: int
    char_end: int

contextual_chunks = []
current_pos = 0

for i, chunk_text in enumerate(raw_chunks):
    # Find position in original content
    chunk_start = raw_content.find(chunk_text, current_pos)
    if chunk_start == -1:
        chunk_start = current_pos  # Fallback
    chunk_end = chunk_start + len(chunk_text)
    
    # Get section context
    context = get_section_context(chunk_start, sections, ARTICLE_TITLE)
    
    contextual_chunks.append(ContextualChunk(
        content=chunk_text,
        chunk_index=i,
        total_chunks=len(raw_chunks),
        breadcrumb=context["breadcrumb"],
        section_title=context["section_title"],
        char_start=chunk_start,
        char_end=chunk_end,
    ))
    
    current_pos = chunk_start + 1

print(f"\nChunks with context:")
for chunk in contextual_chunks[:3]:
    print(f"  Chunk {chunk.chunk_index + 1}: {chunk.breadcrumb}")

Found 71 sections
Split into 63 chunks

Chunks with context:
  Chunk 1: Albert Einstein > Introduction
  Chunk 2: Albert Einstein > Introduction
  Chunk 3: Albert Einstein > Life and career


## Initialize Entity Registry

In [8]:
# Initialize entity registry with article subject
registry = EntityRegistry(source_url=source_url)

# Pre-seed with the article subject
registry.register(
    label=ARTICLE_TITLE,
    entity_type="Person",  # Adjust based on article type
    description=f"Subject of Wikipedia article: {ARTICLE_TITLE}",
    aliases=[ARTICLE_TITLE.split()[-1]],  # Last name as alias
)

print(f"Entity registry initialized with subject: {ARTICLE_TITLE}")
print(f"Subject URI: {registry.entities[registry.normalize_key(ARTICLE_TITLE)]['uri']}")

Entity registry initialized with subject: Albert Einstein
Subject URI: https://en.wikipedia.org/wiki/Albert_Einstein#person_albert_einstein


## Prompt Templates

These prompts are embedded in the generated notebooks for transparency and adjustability.

In [9]:
FACTS_EXTRACTION_PROMPT = """You are an expert at extracting factual information from text.

Given text from a Wikipedia article, extract simple English statements that capture the consensus factual information. Each statement should:
- Be a single, clear sentence
- Contain one main fact or relationship
- Use the full name of entities on first mention
- Be verifiable from the source text
- Avoid opinions, interpretations, or hedged language

The text comes from: {source_url}
Section context: {breadcrumb}

Known entities (use consistent names):
{known_entities}

---
{chunk_text}
---

Extract factual statements as a bulleted list. Also identify any new entities (people, places, organizations, concepts, events, works) that should be added to the registry.
"""

RDF_GENERATION_PROMPT = """You are an expert at converting factual statements to RDF triples in Turtle format.

Convert the following factual statements to RDF using schema.org vocabulary where possible.

Source: {source_url}
Section: {breadcrumb}

Use these prefixes:
{prefixes}

Entity registry (use these URIs):
{entity_registry}

Guidelines:
- Use schema.org properties (schema:birthDate, schema:birthPlace, schema:worksFor, etc.)
- For relationships not in schema.org, use wiki3: prefix
- Include rdfs:label for entities
- Use xsd datatypes for dates and numbers
- Entity URIs should use the source URL as base with fragment identifiers

---
{facts}
---

Generate Turtle RDF:
"""

RDF_PREFIXES = """@prefix schema: <https://schema.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix wiki3: <https://wiki3.ai/vocab/> .
@base <{source_url}> .
"""

print("Prompt templates defined")

Prompt templates defined


## Generate Chunks Notebook

In [ ]:
def generate_chunks_notebook(chunks: list, provenance: dict, registry: EntityRegistry, 
                             llm_config: dict, output_path: str):
    """Generate a notebook with chunked source text and context metadata.
    
    Each chunk cell is followed by a signature raw cell with its CID.
    """
    nb = new_notebook()
    
    # Cell 0: Provenance markdown
    provenance_yaml = f"""# Chunked Text: {provenance['article_title']}

## Provenance

```yaml
source_url: {provenance['source_url']}
article_title: {provenance['article_title']}
fetched_at: {provenance['fetched_at']}
content_length: {provenance['content_length']}
license: {provenance['license']}
license_url: {provenance['license_url']}
attribution: {provenance['attribution']}
chunk_size: {CHUNK_SIZE}
chunk_overlap: {CHUNK_OVERLAP}
total_chunks: {len(chunks)}
generated_by: wiki_to_kg_pipeline.ipynb
generated_at: {datetime.now().isoformat()}
```

## Processing Instructions

Each chunk below contains source text with contextual metadata. The context line (before the separator) provides:
- **Context**: Hierarchical breadcrumb showing article > section path
- **Chunk**: Position in sequence

The text below the `---` separator is the unchanged source content.
Each chunk is followed by a signature cell containing its Content ID (CID).
"""
    nb.cells.append(new_markdown_cell(provenance_yaml))
    
    # Cell 1: Entity registry (raw cell)
    nb.cells.append(new_raw_cell(registry.to_json()))
    
    # Chunk cells with signatures
    # Compute CID of raw source for provenance
    source_cid = compute_cid(provenance['source_url'] + str(provenance['content_length']))
    
    for chunk in chunks:
        # Content cell
        chunk_content = f"""**Context:** {chunk.breadcrumb}
**Chunk:** {chunk.chunk_index + 1} of {chunk.total_chunks}

---

{chunk.content}
"""
        nb.cells.append(new_markdown_cell(chunk_content))
        
        # Signature cell
        chunk_cid = compute_cid(chunk_content)
        signature = make_signature(
            cell_num=chunk.chunk_index + 1,
            cell_type="chunk",
            cid=chunk_cid,
            from_cid=source_cid
        )
        nb.cells.append(new_raw_cell(json.dumps(signature)))
    
    # Write notebook
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)
    
    return output_path

# Generate chunks notebook (simple filename, timestamp is in directory)
# Always regenerate chunks since they come from fresh Wikipedia fetch
chunks_path = os.path.join(RUN_OUTPUT_DIR, "chunks.ipynb")
generate_chunks_notebook(contextual_chunks, provenance, registry, LLM_CONFIG, chunks_path)
print(f"Generated: {chunks_path}")

Generated: data/albert_einstein/20251218_202954/chunks.ipynb


## Generate Facts Notebook (Structure Only)

Creates the facts notebook with placeholders. Actual content is generated in the next step.

In [ ]:
def generate_facts_notebook(chunks: list, provenance: dict, registry: EntityRegistry,
                            llm_config: dict, source_notebook: str, output_path: str):
    """Generate a notebook structure for extracted facts.
    
    Creates header cells only - actual fact content is populated by the pipeline
    during the facts extraction phase.
    
    Args:
        chunks: List of ContextualChunk objects (for count info)
        provenance: Dictionary with article provenance info
        registry: EntityRegistry for consistent URIs
        llm_config: Dict with model, endpoint config
        source_notebook: Relative path to source chunks notebook
        output_path: Where to write the facts notebook
    """
    nb = new_notebook()
    
    # Cell 0: Provenance markdown
    provenance_yaml = f"""# Extracted Facts: {provenance['article_title']}

## Provenance

```yaml
source_notebook: {source_notebook}
source_url: {provenance['source_url']}
article_title: {provenance['article_title']}
license: {provenance['license']}
license_url: {provenance['license_url']}
extraction_model: {llm_config['model']}
extraction_timestamp: {datetime.now().isoformat()}
total_chunks: {len(chunks)}
generated_by: wiki_to_kg_pipeline.ipynb
```

## Processing Instructions

This notebook contains factual statements extracted from source chunks.
Each facts cell is followed by a signature cell with CID provenance linking back to its source chunk.

To regenerate a specific cell: delete both the content cell and its signature, then re-run the pipeline.

Structure:
- **Context**: Section breadcrumb from source
- **Chunk**: Position in sequence
- **Facts**: Bulleted list of extracted factual statements
"""
    nb.cells.append(new_markdown_cell(provenance_yaml))
    
    # Cell 1: Entity registry (raw cell)
    nb.cells.append(new_raw_cell(registry.to_json()))
    
    # Note: Actual fact cells are added by the pipeline during processing
    # Each will be: markdown content cell + raw signature cell
    
    # Write notebook
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)
    
    return output_path

# Generate facts notebook (only if doesn't exist - preserve previous run's content)
facts_path = os.path.join(RUN_OUTPUT_DIR, "facts.ipynb")
if not os.path.exists(facts_path):
    generate_facts_notebook(contextual_chunks, provenance, registry, LLM_CONFIG, 
                           "chunks.ipynb", facts_path)
    print(f"Generated: {facts_path}")
else:
    print(f"Using existing: {facts_path} (from previous run)")

Generated: data/albert_einstein/20251218_202954/facts.ipynb


## Generate RDF Notebook (Structure Only)

Creates the RDF notebook with placeholders. Actual content is generated after facts extraction.

In [ ]:
def generate_rdf_notebook_header(provenance: dict, registry: EntityRegistry,
                                  llm_config: dict, prompt_template: str, prefixes: str) -> nbformat.NotebookNode:
    """Generate just the header cells for RDF notebook."""
    nb = new_notebook()
    
    formatted_prefixes = prefixes.format(source_url=provenance['source_url'])
    provenance_md = f"""# RDF Triples: {provenance['article_title']}

## Provenance

```yaml
source_url: {provenance['source_url']}
article_title: {provenance['article_title']}
license: {provenance['license']}
license_url: {provenance['license_url']}
source_notebook: facts.ipynb
generated_by: wiki_to_kg_pipeline.ipynb
generated_at: {datetime.now().isoformat()}
llm_provider: {llm_config['provider']}
llm_model: {llm_config['model']}
llm_temperature: {llm_config['temperature']}
rdf_format: Turtle
```

## RDF Prefixes

```turtle
{formatted_prefixes}
```

## Processing Instructions

This notebook contains RDF triples in Turtle format, one cell per source facts cell.
Each content cell is followed by a signature cell with CID provenance.

To regenerate a specific cell: delete both the content cell and its signature, then re-run the pipeline.

## Prompt Template

```
{prompt_template}
```
"""
    nb.cells.append(new_markdown_cell(provenance_md))
    nb.cells.append(new_raw_cell(registry.to_json()))
    
    return nb

# Create initial RDF notebook (only if doesn't exist - preserve previous run's content)
rdf_path = os.path.join(RUN_OUTPUT_DIR, "rdf.ipynb")

if not os.path.exists(rdf_path):
    rdf_nb = generate_rdf_notebook_header(provenance, registry, LLM_CONFIG, RDF_GENERATION_PROMPT, RDF_PREFIXES)
    with open(rdf_path, 'w', encoding='utf-8') as f:
        nbformat.write(rdf_nb, f)
    print(f"Created RDF notebook: {rdf_path}")
else:
    print(f"Using existing: {rdf_path} (from previous run)")

Created RDF notebook: data/albert_einstein/20251218_202954/rdf.ipynb


## Process Chunks → Extract Facts

Run the LLM on each chunk to extract factual statements and update the facts notebook.

In [13]:
# Create LLM client with timeout for facts extraction
facts_llm = ChatOpenAI(
    model=LLM_CONFIG["model"],
    temperature=LLM_CONFIG["temperature"],
    base_url=LLM_CONFIG["base_url"],
    api_key="lm-studio",
    timeout=CELL_TIMEOUT_SECONDS,  # HTTP timeout in seconds
    max_retries=0,  # Don't retry on timeout
)

# Create facts extraction prompt
facts_prompt = ChatPromptTemplate.from_template("""You are an expert at extracting factual information from text.

Given text from a Wikipedia article, extract simple English statements that capture the consensus factual information. Each statement should:
- Be a single, clear sentence
- Contain one main fact or relationship
- Use the full name of entities on first mention
- Be verifiable from the source text
- Avoid opinions, interpretations, or hedged language

Source: {source_url}
Section context: {breadcrumb}

Known entities (use consistent names):
{known_entities}

---
{chunk_text}
---

Extract factual statements as a bulleted list:""")

facts_chain = facts_prompt | facts_llm

def get_known_entities_text(registry: EntityRegistry) -> str:
    """Format known entities for prompt context."""
    lines = []
    for entity in registry.entities.values():
        lines.append(f"- {entity['label']} ({entity['type']})")
    return "\n".join(lines) if lines else "None yet"

# Read chunks notebook to get source content and CIDs
log_progress("Reading chunks notebook...")
chunks_nb = nbformat.read(chunks_path, as_version=4)
chunk_signatures = extract_signatures(chunks_nb)

# Build list of chunk content with CIDs
# Chunks notebook structure: [provenance, registry, chunk1, sig1, chunk2, sig2, ...]
chunk_data = []
cell_idx = 2  # Skip provenance and registry
while cell_idx < len(chunks_nb.cells):
    cell = chunks_nb.cells[cell_idx]
    if cell.cell_type == 'markdown':
        content = cell.source
        # Get corresponding signature (next cell)
        sig = None
        if cell_idx + 1 < len(chunks_nb.cells):
            sig = parse_signature(chunks_nb.cells[cell_idx + 1].source)
        
        # Extract breadcrumb
        context_match = re.search(r'\*\*Context:\*\*\s*(.+)', content)
        breadcrumb = context_match.group(1) if context_match else "Unknown"
        
        # Extract chunk text (after ---)
        parts = content.split("---\n", 1)
        chunk_text = parts[1].strip() if len(parts) > 1 else content
        
        chunk_data.append({
            "cell_num": sig["cell"] if sig else len(chunk_data) + 1,
            "content": content,
            "chunk_text": chunk_text,
            "breadcrumb": breadcrumb,
            "cid": sig["cid"] if sig else compute_cid(content),
        })
        cell_idx += 2  # Skip content and signature
    else:
        cell_idx += 1

log_progress(f"Found {len(chunk_data)} chunks with CIDs")

# Read existing facts notebook and extract signatures
facts_nb = nbformat.read(facts_path, as_version=4)
facts_signatures = extract_signatures(facts_nb)

log_progress(f"Found {len(facts_signatures)} existing fact signatures")
log_progress(f"HTTP timeout: {CELL_TIMEOUT_SECONDS}s per request")
log_progress("-" * 50)

# Process each chunk
processed_count = 0
skipped_count = 0
error_count = 0

for chunk in chunk_data:
    cell_num = chunk["cell_num"]
    source_cid = chunk["cid"]
    
    # Check if we already have up-to-date facts for this chunk
    existing_sig = facts_signatures.get(cell_num)
    if existing_sig and existing_sig["from_cid"] == source_cid:
        log_progress(f"  Chunk {cell_num}: ⊘ Up-to-date (CID match), skipping")
        skipped_count += 1
        continue
    
    # Need to generate (or regenerate) this cell
    status = "↻ Regenerating" if existing_sig else "+ Generating"
    chunk_preview = chunk["chunk_text"][:80].replace('\n', ' ')
    log_progress(f"  Chunk {cell_num}: {status}")
    log_progress(f"    Context: {chunk['breadcrumb']}")
    log_progress(f"    Input: {chunk_preview}...")
    log_progress(f"    [Calling LLM...]", end=" ")
    
    start_time = time.time()
    
    # Call LLM to extract facts
    try:
        result = facts_chain.invoke({
            "source_url": provenance["source_url"],
            "breadcrumb": chunk["breadcrumb"],
            "known_entities": get_known_entities_text(registry),
            "chunk_text": chunk["chunk_text"],
        })
        facts_content = result.content
        elapsed = time.time() - start_time
        log_progress(f"✓ ({len(facts_content)} chars, {elapsed:.1f}s)")
        processed_count += 1
    except Exception as e:
        elapsed = time.time() - start_time
        error_type = type(e).__name__
        error_msg = str(e)[:100]
        facts_content = f"# Error: {error_type}: {e}"
        log_progress(f"✗ {error_type} after {elapsed:.1f}s")
        log_progress(f"    Error: {error_msg}")
        error_count += 1
    
    # Build the facts cell content
    facts_cell_content = f"""**Context:** {chunk['breadcrumb']}
**Chunk:** {cell_num} of {len(chunk_data)}

---

{facts_content}
"""
    facts_cid = compute_cid(facts_cell_content)
    signature = make_signature(cell_num, "facts", facts_cid, source_cid)
    
    # Find where to insert/update in the notebook
    # If there's an existing signature for this cell, find and remove old content+sig
    if existing_sig:
        # Find and remove the old cells
        new_cells = [facts_nb.cells[0], facts_nb.cells[1]]  # Keep header
        i = 2
        while i < len(facts_nb.cells):
            cell = facts_nb.cells[i]
            if cell.cell_type == 'raw':
                sig = parse_signature(cell.source)
                if sig and sig["cell"] == cell_num:
                    # Skip this signature and its preceding content cell
                    i += 1
                    continue
            # Check if this is content for the cell we're replacing
            if i > 0 and i + 1 < len(facts_nb.cells):
                next_sig = parse_signature(facts_nb.cells[i + 1].source) if facts_nb.cells[i + 1].cell_type == 'raw' else None
                if next_sig and next_sig["cell"] == cell_num:
                    i += 2  # Skip content and signature
                    continue
            new_cells.append(cell)
            i += 1
        facts_nb.cells = new_cells
    
    # Append new content and signature
    facts_nb.cells.append(new_markdown_cell(facts_cell_content))
    facts_nb.cells.append(new_raw_cell(json.dumps(signature)))
    
    # Update signatures dict
    facts_signatures[cell_num] = signature
    
    # Save notebook after each cell
    log_progress(f"    [Saving notebook...]", end=" ")
    with open(facts_path, 'w', encoding='utf-8') as f:
        nbformat.write(facts_nb, f)
    log_progress("saved")

log_progress("-" * 50)
log_progress(f"Facts extraction complete:")
log_progress(f"  - {processed_count} generated")
log_progress(f"  - {skipped_count} skipped (up-to-date)")
log_progress(f"  - {error_count} errors/timeouts")

Reading chunks notebook...
Found 63 chunks with CIDs
Found 0 existing fact signatures
HTTP timeout: 60s per request
--------------------------------------------------
  Chunk 1: + Generating
    Context: Albert Einstein > Introduction
    Input: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical ph...
    [Calling LLM...] ✓ (2078 chars, 6.4s)
    [Saving notebook...] saved
  Chunk 2: + Generating
    Context: Albert Einstein > Introduction
    Input: In 1905, sometimes described as his annus mirabilis (miracle year), he published...
    [Calling LLM...] ✓ (1715 chars, 4.4s)
    [Saving notebook...] saved
  Chunk 3: + Generating
    Context: Albert Einstein > Life and career
    Input: == Life and career ==   === Childhood, youth and education ===  Albert Einstein ...
    [Calling LLM...] ✓ (1116 chars, 3.5s)
    [Saving notebook...] saved
  Chunk 4: + Generating
    Context: Albert Einstein > Life and career > Childhood, youth and education
    Input: In 1894

In [14]:
# Summary of facts extraction (notebook already updated incrementally)
print(f"Facts notebook: {facts_path}")
print(f"  - Updated incrementally during processing")
print(f"  - Ready for review/editing before RDF generation")

Facts notebook: data/albert_einstein/20251218_202954/facts.ipynb
  - Updated incrementally during processing
  - Ready for review/editing before RDF generation


## Process Facts → Generate RDF

Run the LLM on each facts cell to generate RDF triples and update the RDF notebook.

In [15]:
# Load schema matcher with pre-built vocabulary index
from schema_matcher import SchemaMatcher
from langchain_core.tools import tool
from typing import List

VOCAB_CACHE_DIR = "data/vocab_cache"
schema_matcher = SchemaMatcher.load(VOCAB_CACHE_DIR, embed_base_url=LLM_CONFIG["base_url"])
log_progress(f"Loaded schema matcher with {len(schema_matcher.vocabularies)} vocabularies")

# Collector for emitted triples (reset per cell)
emitted_triples: List[dict] = []

# Iteration tracking for statistics
iteration_counts: List[int] = []  # Track iterations per cell

# === Schema lookup tools ===

@tool
def find_rdf_class(description: str) -> str:
    """Find the best schema.org class/type for an entity based on a natural language description.
    
    Use this when you need to determine the rdf:type of an entity.
    Example: find_rdf_class("a person who does scientific research")
    
    Args:
        description: Natural language description of the entity type
    
    Returns:
        Top matching schema.org classes with URIs and descriptions
    """
    results = schema_matcher.find_class(description, top_k=5)
    if not results:
        return "No matches found. Use a generic type like schema:Thing"
    
    lines = ["Top matching classes:"]
    for r in results:
        lines.append(f"  {r['prefix']} ({r['score']:.2f})")
        lines.append(f"    URI: {r['uri']}")
        if r['description']:
            lines.append(f"    Description: {r['description'][:100]}")
    return "\n".join(lines)

@tool
def find_rdf_property(description: str, subject_type: str = "", object_type: str = "") -> str:
    """Find the best schema.org property/predicate for a relationship.
    
    Use this when you need to find the right predicate for a triple.
    Example: find_rdf_property("the date when someone was born", subject_type="Person")
    
    Args:
        description: Natural language description of the relationship
        subject_type: Optional - the type of the subject (e.g., "Person", "Organization")  
        object_type: Optional - the type of the object/value (e.g., "Date", "Place")
    
    Returns:
        Top matching schema.org properties with URIs, domains, and ranges
    """
    results = schema_matcher.find_property(
        description, 
        subject_type=subject_type or None,
        object_type=object_type or None,
        top_k=5
    )
    if not results:
        return "No matches found. Consider using rdfs:label or a descriptive URI fragment."
    
    lines = ["Top matching properties:"]
    for r in results:
        lines.append(f"  {r['prefix']} ({r['score']:.2f})")
        lines.append(f"    URI: {r['uri']}")
        if r['domain']:
            lines.append(f"    Domain: {r['domain']}")
        if r['range']:
            lines.append(f"    Range: {r['range']}")
        if r['description']:
            lines.append(f"    Description: {r['description'][:80]}")
    return "\n".join(lines)

# === Triple output tools ===

@tool
def emit_triple(subject: str, predicate: str, object_value: str) -> str:
    """Emit a single RDF triple. Use this to output each triple you generate.
    
    Args:
        subject: The subject URI (e.g., "<https://example.org#entity>") or prefixed (e.g., "schema:Person")
        predicate: The predicate URI or prefixed term (e.g., "schema:birthDate", "rdf:type")
        object_value: The object - a URI, prefixed term, or literal with datatype 
                      (e.g., '"1879-03-14"^^xsd:date', '"Albert Einstein"@en', '<https://...>')
    
    Returns:
        Confirmation message
    """
    emitted_triples.append({
        "subject": subject,
        "predicate": predicate,
        "object": object_value,
    })
    return f"Triple recorded: {subject} {predicate} {object_value}"

@tool  
def emit_triples(triples: List[dict]) -> str:
    """Emit multiple RDF triples at once. More efficient than calling emit_triple repeatedly.
    
    Args:
        triples: List of triple dictionaries, each with keys:
                 - subject: The subject URI or prefixed term
                 - predicate: The predicate URI or prefixed term  
                 - object: The object URI, prefixed term, or literal
                 
    Example:
        emit_triples([
            {"subject": "<#person_einstein>", "predicate": "rdf:type", "object": "schema:Person"},
            {"subject": "<#person_einstein>", "predicate": "schema:name", "object": '"Albert Einstein"'},
            {"subject": "<#person_einstein>", "predicate": "schema:birthDate", "object": '"1879-03-14"^^xsd:date'}
        ])
    
    Returns:
        Confirmation with count of triples recorded
    """
    count = 0
    for t in triples:
        if isinstance(t, dict) and all(k in t for k in ("subject", "predicate", "object")):
            emitted_triples.append({
                "subject": t["subject"],
                "predicate": t["predicate"],
                "object": t["object"],
            })
            count += 1
    return f"Recorded {count} triples"

# Create LLM client with timeout
rdf_llm = ChatOpenAI(
    model=LLM_CONFIG["model"],
    temperature=LLM_CONFIG["temperature"],
    base_url=LLM_CONFIG["base_url"],
    api_key="lm-studio",  # type: ignore
    timeout=CELL_TIMEOUT_SECONDS,
    max_retries=0,
)

# Bind all tools to the LLM
rdf_tools = [find_rdf_class, find_rdf_property, emit_triple, emit_triples]
rdf_llm_with_tools = rdf_llm.bind_tools(rdf_tools)

# Create RDF generation prompt
rdf_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at converting factual statements to RDF triples.

You have access to these tools:

LOOKUP TOOLS (use first to find correct vocabulary):
- find_rdf_class: Find the best schema.org class/type for an entity
- find_rdf_property: Find the best predicate for a relationship

OUTPUT TOOLS (use to emit your triples):
- emit_triple: Output a single triple (subject, predicate, object)
- emit_triples: Output multiple triples at once (more efficient)

WORKFLOW:
1. First use find_rdf_class and find_rdf_property to look up appropriate schema.org terms
2. Then use emit_triple or emit_triples to output each RDF triple
3. Finally, provide a brief summary noting any quality concerns

IMPORTANT:
- Do NOT write Turtle syntax in your response - use the emit tools instead
- Use schema.org terms you looked up (e.g., schema:Person, schema:birthDate)
- For URIs, use angle brackets: <https://...> or fragment references: <#entity_id>
- For literals, use quotes with optional datatype: "value"^^xsd:date or "text"@en
- For prefixed terms as objects, just use the prefix: schema:Person"""),
    ("human", """Convert these factual statements to RDF triples.

Source: {source_url}
Section context: {breadcrumb}

Entity Registry (use these URIs for known entities):
{entity_registry}

Factual statements to convert:
{facts}

Use the lookup tools to find schema.org terms, then emit your triples. End with a brief summary."""),
])

def triples_to_turtle(triples: List[dict]) -> str:
    """Convert collected triples to Turtle format."""
    if not triples:
        return "# No triples emitted"
    
    lines = []
    for t in triples:
        lines.append(f"{t['subject']} {t['predicate']} {t['object']} .")
    return "\n".join(lines)

# Tool-calling loop that collects emitted triples
def call_llm_with_tools(prompt_vars: dict, max_iterations: int = MAX_ITERATIONS) -> tuple[str, List[dict], int]:
    """Call LLM with tool support. Returns (summary_response, collected_triples, iterations_used)."""
    global emitted_triples
    emitted_triples = []  # Reset collector
    
    messages = rdf_prompt.format_messages(**prompt_vars)
    
    for iteration in range(max_iterations):
        response = rdf_llm_with_tools.invoke(messages)
        
        # Check if there are tool calls
        if hasattr(response, 'tool_calls') and response.tool_calls:
            # Add assistant message with tool calls
            messages.append(response)
            
            # Execute each tool call
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                
                # Find and execute the tool
                if tool_name == 'find_rdf_class':
                    tool_result = find_rdf_class.invoke(tool_args)
                elif tool_name == 'find_rdf_property':
                    tool_result = find_rdf_property.invoke(tool_args)
                elif tool_name == 'emit_triple':
                    tool_result = emit_triple.invoke(tool_args)
                elif tool_name == 'emit_triples':
                    tool_result = emit_triples.invoke(tool_args)
                else:
                    tool_result = f"Unknown tool: {tool_name}"
                
                # Add tool result message
                from langchain_core.messages import ToolMessage
                messages.append(ToolMessage(content=str(tool_result), tool_call_id=tool_call['id']))
        else:
            # No tool calls, return the summary and collected triples
            summary = response.content if hasattr(response, 'content') else str(response)
            return summary, list(emitted_triples), iteration + 1
    
    # Max iterations reached
    summary = response.content if hasattr(response, 'content') else "Max iterations reached"
    return summary, list(emitted_triples), max_iterations

def format_entity_registry_for_prompt(registry: EntityRegistry) -> str:
    """Format registry for RDF prompt."""
    lines = []
    for entity in registry.entities.values():
        lines.append(f"<{entity['uri']}> # {entity['label']} ({entity['type']})")
    return "\n".join(lines) if lines else "# No entities registered yet"

# Read facts notebook to get source content and CIDs
log_progress("Reading facts notebook...")
facts_nb = nbformat.read(facts_path, as_version=4)
facts_signatures = extract_signatures(facts_nb)

# Build list of facts content with CIDs
facts_data = []
cell_idx = 2  # Skip provenance and registry
while cell_idx < len(facts_nb.cells):
    cell = facts_nb.cells[cell_idx]
    if cell.cell_type == 'markdown':
        content = cell.source
        # Get corresponding signature (next cell)
        sig = None
        if cell_idx + 1 < len(facts_nb.cells):
            sig = parse_signature(facts_nb.cells[cell_idx + 1].source)
        
        # Extract breadcrumb
        context_match = re.search(r'\*\*Context:\*\*\s*(.+)', content)
        breadcrumb = context_match.group(1) if context_match else "Unknown"
        
        # Extract facts (after ---)
        parts = content.split("---\n", 1)
        facts_text = parts[1].strip() if len(parts) > 1 else content
        
        facts_data.append({
            "cell_num": sig["cell"] if sig else len(facts_data) + 1,
            "content": content,
            "facts_text": facts_text,
            "breadcrumb": breadcrumb,
            "cid": sig["cid"] if sig else compute_cid(content),
        })
        cell_idx += 2  # Skip content and signature
    else:
        cell_idx += 1

log_progress(f"Found {len(facts_data)} facts cells with CIDs")

# Read existing RDF notebook and extract signatures
rdf_nb = nbformat.read(rdf_path, as_version=4)
rdf_signatures = extract_signatures(rdf_nb)

log_progress(f"Found {len(rdf_signatures)} existing RDF signatures")
log_progress(f"HTTP timeout: {CELL_TIMEOUT_SECONDS}s per request")
log_progress(f"Max iterations: {MAX_ITERATIONS} per cell")
log_progress("-" * 50)

# Process each facts cell
processed_count = 0
skipped_count = 0
error_count = 0
total_triples = 0
iteration_counts = []  # Reset for this run
max_iterations_hit = 0  # Count cells that hit the limit

for facts_item in facts_data:
    cell_num = facts_item["cell_num"]
    source_cid = facts_item["cid"]
    
    # Skip cells that had errors in facts extraction
    if facts_item["facts_text"].startswith("# Error:"):
        log_progress(f"  Facts {cell_num}: ⊘ Source had error, skipping")
        skipped_count += 1
        continue
    
    # Check if we already have up-to-date RDF for these facts
    existing_sig = rdf_signatures.get(cell_num)
    if existing_sig and existing_sig["from_cid"] == source_cid:
        log_progress(f"  Facts {cell_num}: ⊘ Up-to-date (CID match), skipping")
        skipped_count += 1
        continue
    
    # Need to generate (or regenerate) this cell
    status = "↻ Regenerating" if existing_sig else "+ Generating"
    facts_preview = facts_item["facts_text"][:80].replace('\n', ' ')
    log_progress(f"  Facts {cell_num}: {status}")
    log_progress(f"    Context: {facts_item['breadcrumb']}")
    log_progress(f"    Input: {facts_preview}...")
    log_progress(f"    [Calling LLM with tools...]", end=" ")
    
    start_time = time.time()
    
    # Call LLM with tool support - now returns (summary, triples, iterations)
    try:
        summary, triples, iterations_used = call_llm_with_tools({
            "source_url": provenance["source_url"],
            "breadcrumb": facts_item["breadcrumb"],
            "entity_registry": format_entity_registry_for_prompt(registry),
            "facts": facts_item["facts_text"],
        })
        elapsed = time.time() - start_time
        triple_count = len(triples)
        total_triples += triple_count
        iteration_counts.append(iterations_used)
        
        # Check if we hit the limit
        if iterations_used >= MAX_ITERATIONS:
            max_iterations_hit += 1
            log_progress(f"⚠ ({triple_count} triples, {iterations_used} iters [MAX], {elapsed:.1f}s)")
        else:
            log_progress(f"✓ ({triple_count} triples, {iterations_used} iters, {elapsed:.1f}s)")
        
        # Convert triples to Turtle
        rdf_content = triples_to_turtle(triples)
        
        # Include summary as comment if it has content
        if summary and summary.strip():
            # Clean up summary for comment
            summary_lines = summary.strip().split('\n')
            summary_comment = '\n'.join(f"# {line}" for line in summary_lines[:5])
            rdf_content = f"{summary_comment}\n\n{rdf_content}"
        
        processed_count += 1
    except Exception as e:
        elapsed = time.time() - start_time
        error_type = type(e).__name__
        error_msg = str(e)[:100]
        rdf_content = f"# Error: {error_type}: {e}"
        log_progress(f"✗ {error_type} after {elapsed:.1f}s")
        log_progress(f"    Error: {error_msg}")
        error_count += 1
    
    # Build the RDF cell content
    rdf_cell_content = f"""# Context: {facts_item['breadcrumb']}
# Cell: {cell_num} of {len(facts_data)}

{rdf_content}
"""
    rdf_cid = compute_cid(rdf_cell_content)
    signature = make_signature(cell_num, "rdf", rdf_cid, source_cid)
    
    # Find where to insert/update in the notebook
    if existing_sig:
        # Find and remove the old cells
        new_cells = [rdf_nb.cells[0], rdf_nb.cells[1]]  # Keep header
        i = 2
        while i < len(rdf_nb.cells):
            cell = rdf_nb.cells[i]
            if cell.cell_type == 'raw':
                sig = parse_signature(cell.source)
                if sig and sig["cell"] == cell_num:
                    i += 1
                    continue
            if i > 0 and i + 1 < len(rdf_nb.cells):
                next_sig = parse_signature(rdf_nb.cells[i + 1].source) if rdf_nb.cells[i + 1].cell_type == 'raw' else None
                if next_sig and next_sig["cell"] == cell_num:
                    i += 2
                    continue
            new_cells.append(cell)
            i += 1
        rdf_nb.cells = new_cells
    
    # Append new content and signature
    rdf_nb.cells.append(new_raw_cell(rdf_cell_content))
    rdf_nb.cells.append(new_raw_cell(json.dumps(signature)))
    
    # Update signatures dict
    rdf_signatures[cell_num] = signature
    
    # Save notebook after each cell
    log_progress(f"    [Saving notebook...]", end=" ")
    with open(rdf_path, 'w', encoding='utf-8') as f:
        nbformat.write(rdf_nb, f)
    log_progress("saved")

log_progress("-" * 50)
log_progress(f"RDF generation complete:")
log_progress(f"  - {processed_count} cells processed")
log_progress(f"  - {total_triples} total triples emitted")
log_progress(f"  - {skipped_count} skipped (up-to-date)")
log_progress(f"  - {error_count} errors/timeouts")

# Iteration statistics
if iteration_counts:
    log_progress(f"\nIteration statistics (max={MAX_ITERATIONS}):")
    log_progress(f"  - Min: {min(iteration_counts)}")
    log_progress(f"  - Max: {max(iteration_counts)}")
    log_progress(f"  - Mean: {sum(iteration_counts)/len(iteration_counts):.1f}")
    log_progress(f"  - Hit limit: {max_iterations_hit} cells")
    
    # Distribution buckets
    buckets = {"1-5": 0, "6-10": 0, "11-20": 0, "21-50": 0, "51-100": 0, "100+": 0}
    for count in iteration_counts:
        if count <= 5:
            buckets["1-5"] += 1
        elif count <= 10:
            buckets["6-10"] += 1
        elif count <= 20:
            buckets["11-20"] += 1
        elif count <= 50:
            buckets["21-50"] += 1
        elif count <= 100:
            buckets["51-100"] += 1
        else:
            buckets["100+"] += 1
    
    log_progress(f"  - Distribution: {buckets}")

Loaded schema matcher with 1 vocabularies
Reading facts notebook...
Found 63 facts cells with CIDs
Found 0 existing RDF signatures
HTTP timeout: 60s per request
Max iterations: 150 per cell
--------------------------------------------------
  Facts 1: + Generating
    Context: Albert Einstein > Introduction
    Input: • Albert Einstein was born on 14 March 1879 in the German Empire. • Albert Einst...
    [Calling LLM with tools...] ✓ (37 triples, 5 iters, 51.6s)
    [Saving notebook...] saved
  Facts 2: + Generating
    Context: Albert Einstein > Introduction
    Input: • In 1905, Albert Einstein published four groundbreaking papers that outlined a ...
    [Calling LLM with tools...] ✓ (34 triples, 5 iters, 43.9s)
    [Saving notebook...] saved
  Facts 3: + Generating
    Context: Albert Einstein > Life and career
    Input: • Albert Einstein was born in Ulm, in the Kingdom of Württemberg in the German E...
    [Calling LLM with tools...] ✓ (24 triples, 4 iters, 22.9s)
    [Saving note

KeyboardInterrupt: 

In [ ]:
# Summary of RDF generation (notebook already updated incrementally)
print(f"RDF notebook: {rdf_path}")
print(f"  - Updated incrementally during processing")
print(f"  - Ready for review/editing before final export")

RDF notebook: data/albert_einstein_rdf.ipynb
  - Updated incrementally during processing
  - Ready for review/editing before final export


## Export Combined RDF

Combine all RDF cells into a single Turtle file with prefixes.

In [ ]:
# Combine all RDF into a single Turtle file
all_triples = []

# Read the updated RDF notebook
rdf_nb = nbformat.read(rdf_path, as_version=4)

# Collect RDF from all raw cells (skip provenance, registry, and signature cells)
for cell in rdf_nb.cells[2:]:
    if cell.cell_type == 'raw':
        content = cell.source.strip()
        
        # Skip signature cells (JSON objects)
        if content.startswith('{') and '"cid"' in content:
            continue
        
        # Skip empty or error-only cells
        if not content or content.startswith('# Error:'):
            continue
        
        # Skip comment-only cells
        lines = [line for line in content.split('\n') 
                 if line.strip() and not line.strip().startswith('#')]
        if lines:
            all_triples.append(content)

# Format prefixes for the Turtle file
turtle_prefixes = RDF_PREFIXES.format(source_url=provenance['source_url'])

# Build complete Turtle file
turtle_output = f"""# RDF Knowledge Graph: {provenance['article_title']}
# Source: {provenance['source_url']}
# License: {provenance['license']}
# Generated: {datetime.now().isoformat()}

{turtle_prefixes}

# === Triples ===

"""

turtle_output += "\n\n".join(all_triples)

# Save to file with simple filename (timestamp is in directory)
turtle_path = os.path.join(RUN_OUTPUT_DIR, "triples.ttl")
with open(turtle_path, 'w', encoding='utf-8') as f:
    f.write(turtle_output)

print(f"Exported RDF to: {turtle_path}")
print(f"  - {len(all_triples)} chunks of triples")
print(f"  - {len(turtle_output)} characters total")

Exported RDF to: data/albert_einstein.ttl
  - 57 chunks of triples
  - 198405 characters total


## Save Initial Entity Registry

In [ ]:
# Save entity registry to JSON file with simple filename (timestamp is in directory)
registry_path = os.path.join(RUN_OUTPUT_DIR, "registry.json")
with open(registry_path, 'w', encoding='utf-8') as f:
    f.write(registry.to_json())

print(f"Saved: {registry_path}")
print(f"\nInitial entities: {len(registry.entities)}")
for key, entity in registry.entities.items():
    print(f"  - {entity['label']} ({entity['type']}): {entity['uri']}")

Saved: data/entity_registry.json

Initial entities: 1
  - Albert Einstein (Person): https://en.wikipedia.org/wiki/Albert_Einstein#person_albert_einstein


## Pipeline Summary

In [ ]:
print("="*60)
print("PIPELINE COMPLETE")
print("="*60)
print(f"\nArticle: {ARTICLE_TITLE}")
print(f"Source: {source_url}")
print(f"License: {provenance['license']}")
print(f"Run timestamp: {RUN_TIMESTAMP}")
print(f"\nOutput directory: {RUN_OUTPUT_DIR}")
print(f"\nGenerated artifacts:")
print(f"  1. chunks.ipynb")
print(f"     - {len(contextual_chunks)} chunks with breadcrumb context")
print(f"  2. facts.ipynb")
print(f"     - Facts extracted from chunks")
print(f"  3. rdf.ipynb")
print(f"     - RDF triples generated via tool calls")
print(f"  4. triples.ttl")
print(f"     - Combined Turtle file for import")
print(f"  5. registry.json")
print(f"     - Entity registry snapshot")
print(f"\nThe intermediate notebooks can be reviewed and edited before re-export.")
print(f"Files can reference each other with relative paths within the run directory.")

PIPELINE COMPLETE

Article: Albert Einstein
Source: https://en.wikipedia.org/wiki/Albert_Einstein
License: CC BY-SA 4.0

Generated artifacts:
  1. data/albert_einstein_chunks.ipynb
     - 63 chunks with breadcrumb context
  2. data/albert_einstein_facts.ipynb
     - Facts extracted from chunks (see notebook for details)
  3. data/albert_einstein_rdf.ipynb
     - RDF triples generated from facts (see notebook for details)
  4. data/albert_einstein.ttl
     - Combined Turtle file for import

Entity registry: data/entity_registry.json

The intermediate notebooks can be reviewed and edited before re-export.
